In [1]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 48.6 MB/s eta 0:00:00


In [2]:
from pulp import *

In [3]:
# Tworzymy problem optymalizacji prądu
prob2 = LpProblem("Optymalizacja przepływu prądu", LpMaximize)

# Definicja zmiennych prądu przez rezystory, ograniczając maksymalny prąd na podstawie danych
I1 = LpVariable("I1", 0, 2)  # maksymalnie 2 A dla I1
I2 = LpVariable("I2", 0, 3)  # maksymalnie 3 A dla I2
I3 = LpVariable("I3", 0, 4)  # maksymalnie 4 A dla I3
I4 = LpVariable("I4", 0, 2)  # maksymalnie 2 A dla I4
I5 = LpVariable("I5", 0, 2)  # maksymalnie 2 A dla I5

# Definicja wartości rezystorów
R1, R2, R3, R4, R5 = 8, 6, 4, 10, 8  # Ohmy

# Dodajemy ograniczenia na podstawie połączeń równoległych i szeregowych

prob2 += I3

prob2 += I1 + I2 == I3
prob2 += I4 + I5 == I3

prob2 += I1 * R1 <= I3 * R3
prob2 += I2 * R2 <= I3 * R3
prob2 += I4 * R4 <= I3 * R5
prob2 += I5 * R5 <= I3 * R5

prob2 += I1*R1 - I2*R2 == 0
prob2 += I4*R4 - I5*R5 == 0

# Rozwiązujemy problem
prob2.solve()

# Wyświetlenie wyników
print("Wyniki optymalizacji prądu w układzie:")
for v in prob2.variables():
    print(f"{v.name}: {v.varValue:.2f} A")
print(f"Maksymalny prąd przez układ wynosi {I3.varValue} A.")

Wyniki optymalizacji prądu w układzie:
I1: 1.54 A
I2: 2.06 A
I3: 3.60 A
I4: 1.60 A
I5: 2.00 A
Maksymalny prąd przez układ wynosi 3.6 A.


/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [4]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum

# Tworzenie modelu
model = LpProblem("Minimalizacja_mocy_rozproszenia", LpMinimize)

# Dane
U = [6, 10, 4, 7, 3]  # napięcia na rezystorach
I = [4, 2, 2, 2, 4]   # prądy w mA (ale do kodu wejdą jako zmienne decyzyjne)
delta_I = 1
# Zmienne decyzyjne - rezystancje R1, R2, R3, R4, R5
G = [LpVariable(f"G{i+1}", lowBound=0.01) for i in range(5)]  # Minimalna wartość > 0, aby uniknąć dzielenia przez zero

# Funkcja celu: minimalizacja całkowitej mocy rozproszonej
model += lpSum((U[i]**2) * G[i] for i in range(5))

# Ograniczenia na rezystory wynikające z prawa Ohma: U_i = I_i * R_i
for i in range(5):
  model += I[i] - delta_I <= U[i]*G[i]
  model += U[i] * G[i] <= I[i] + delta_I

# Rozwiązanie problemu
model.solve()

# Wyświetlenie wyników
print("Minimalna moc rozproszona:", sum([G[i].varValue * U[i]**2 for i in range(5)]), "mW")
print("Optymalne rezystancje:")
for i in range(5):
    print(f"R{i+1} = {1/G[i].varValue} Ω")

Minimalna moc rozproszona: 47.99999986 mW
Optymalne rezystancje:
R1 = 2.0 Ω
R2 = 10.0 Ω
R3 = 4.0 Ω
R4 = 7.000000140000003 Ω
R5 = 1.0 Ω
